# Labolatorium 7
## Dekompozycja spektralna

Oh boy, back to algebra

### Wstęp teoretyczny

Co jak co przydałby się.

Dekompozycja spektralna jest to specjalny przypadek `Dekompozycji na wektory własne` [(ang. Eigendecomposition of a matrix) tak jak nazwa angielska wskazuje jest to podział macierzy (kwadratowej) do postaci kanonicznej, czyli reprezentowanej przez jej `wektory i wartości własne`, więcej [tu](https://en.wikipedia.org/wiki/Eigendecomposition_of_a_matrix)], w której macierz faktoryzowana jest [normalna](https://en.wikipedia.org/wiki/Normal_matrix), lub [symetryczna](https://en.wikipedia.org/wiki/Symmetric_matrix). Terminologia ta wywodzi się z Twierdzenia spektralnego (dlaczego to jest twierdzeniem po polsku kiedy po angielsku to tylko teoria wiedzą tylko najwięksi matematycy)

### 1. Metoda potęgowa

[nawet znalazłem coś na wikipedii, nie tylko na wykładzie z macierzowych xd](https://en.wikipedia.org/wiki/Power_iteration)

Metoda potęgowa ma na celu wyznaczenie dominującej wartości (i wektora) własnej (czyli najbardziej oddalonej od zera, podobny koncept co przy `Singular Value Decomposition`).

Do wyliczenia będziemy stosować wzór rekurencyjny

$$
x_{i+1} = \frac{Ax_i}{||x_i||}
$$

Zapis ten oznacza mnożenie wektora $x_i$ przez macierz A a następnie normalizowanie go po każdym kroku

Proces ten wykonujemy aż do przekroczenia maksymalnej liczby iteracji lub w przypadku przekroczenia kryterium małej poprawki.

In [ ]:
import math
import numpy as np
from copy import deepcopy
import scipy
import scipy.linalg

def normalise_vector(x):
    length = calculate_vector_length(x)

    for i in range(len(x)):
        x[i] /= length

def calculate_vector_length(x):
    length = 0
    for xi in x:
        length += xi**2

    return math.sqrt(length)

def determine_vector_lenght(x1,x2):
    sumVec = []
    for i in range(len(x1)):
        sumVec.append(x1[i]-x2[i])
    
    return calculate_vector_length(sumVec) 

def power_method(matrix, max_iter=10000000, eps=1e-30):
    
    x = np.random.rand(matrix.shape[1])

    for _ in range(max_iter):
        prev_x = deepcopy(x)
        x = matrix.dot(x)
        normalise_vector(x)
        #kryterium małej poprawki
        if abs(determine_vector_lenght(prev_x,x)) < eps:
            return x , max(x, key=lambda x: abs(x))
        
    return x , max(x, key=lambda x: abs(x)) 
    